In [0]:
# Queue 클래스

class Queue:
    def __init__(self):
        self.q = []
        
    def enQueue(self, item):
        self.q.append(item)
        
    def deQueue(self):
        if self.isEmpty() == False: 
            return self.q.pop(0)
    
    def size(self):
        return len(self.q)
    
    def isEmpty(self):
        if len(self.q) > 0:
            return False
        else:
            return True
        
    def peek(self):
        if self.isEmpty() == False: 
            return self.q[0]
    
    def delete(self, item):
        if item in self.q: 
            self.q.remove(item)
        else:
            print("해당 아이템이 존재하지 않습니다.")
            
    def lastPeek(self):
        if self.isEmpty() == False:
            return self.q[-1]

In [0]:
class Cust:
    def __init__(self, curTime, mean = 1.0):
        if curTime > 240 and curTime < 300:
            self.arriveTime = curTime + np.random.exponential(scale = mean/2)
        else:
            self.arriveTime = curTime + np.random.exponential(scale = mean)
        self.orderTime = 0
        self.outTime = 0
    # arriveTime, orderTime, outTime 선언
    
    def setTime(self, time):
        self.orderTime = time
        _tmp = np.random.normal(1, 0.2)
        if _tmp < 0 : _tmp = 0
        self.outTime = self.orderTime + _tmp
    # 외부로부터 time을 받아서 orderTime 설정(orderTime은 Cust 단독만으로 판단 X)
    # orderTime + cookTime을 통해 outTIme 설정
    
class Shop:
    def __init__(self, n):
        self.n = n
        self.custQueue = []
        for i in range(n):
            self.custQueue.append(Queue())
    # 점원의 수를 n으로 정의하고 n개의 큐 생성
    # self.custQueue가 하나의 큰 가게가 된다.
    
    def getnCust(self):
        size = []
        for i in range(self.n):
            size.append(self.custQueue[i].size())
        return size
    # 각 점원마다 대기하고 있는 손님의 수 반환
    
    def entCust(self, cust):
        size = np.array(self.getnCust())
        minQIdx = np.argmin(size) # np.argmin(list) : list 값들 중 최솟값의 인덱스 반환
        lastCust = self.custQueue[minQIdx].lastPeek() # 마지막 손님 추출
        if lastCust == None:
            cust.setTime(cust.arriveTime)
        # 큐에 손님이 없으면 현재손님의 도착시간으로 시간 설정
        else:
            cust.setTime(max(cust.arriveTime, lastCust.outTime))
        # 큐에 손님이 있으면 현재손님의 도착시간과 마지막손님의 나가는 시간을 비교하여 시간 설정
        self.custQueue[minQIdx].enQueue(cust) # 손님 입장
        return minQIdx
        
    def outCust(self, curTime):
        for i in range(self.n):
            while self.custQueue[i].size() > 0 and self.custQueue[i].peek().outTime < curTime:
                self.custQueue[i].deQueue()
    # 각 점원에 대해서 큐에 손님이 있고 가장 오래 있었던 손님의 outTime 보다
    # curTime이 지나있으면 손님을 큐에서 삭제
    # 비교/삭제 과정을 반복

In [0]:
import numpy as np
np.random.seed(seed=1)
curTime = 0
s = Shop(2)
qMax = 0 # 지금까지 큐에 들어있었던 최대 인원
while curTime < 14*60:
    cust = Cust(curTime) # 첫번째 손님 생성
    curTime = cust.arriveTime # 현재시간 업데이트
    qNo = s.entCust(cust) # qNo : 몇 번째 큐에 입장했는지에 대한 정보
    s.outCust(curTime) # 현재시간을 이용하여 나가는 손님 업데이트
    if type(s.getnCust()) == int: # 점원을 1명으로 설정했을 경우
        nCust = s.getnCust()
    else:  # 점원을 여러명으로 설정했을 경우
        nCust = max(s.getnCust())
    if qMax < nCust:
        qMax = nCust
    print("%7.2f -> %7.2f -> %7.2f " %(cust.arriveTime, cust.orderTime, cust.outTime), s.getnCust(),qNo)
print("Max Queue : %d" % qMax)

   0.54 ->    0.54 ->    1.38  [1, 0] 0
   0.64 ->    0.64 ->    1.55  [1, 1] 1
   0.84 ->    1.38 ->    2.16  [2, 1] 0
   1.62 ->    1.62 ->    2.29  [1, 1] 1
   2.16 ->    2.16 ->    2.96  [1, 1] 0
   4.26 ->    4.26 ->    5.39  [1, 0] 0
   4.29 ->    4.29 ->    5.12  [1, 1] 1
   5.11 ->    5.39 ->    6.75  [2, 1] 0
   5.26 ->    5.26 ->    6.37  [2, 1] 1
   8.71 ->    8.71 ->    9.60  [0, 1] 1
   9.09 ->    9.09 ->   10.23  [1, 1] 0
  11.34 ->   11.34 ->   12.41  [1, 0] 0
  11.43 ->   11.43 ->   12.53  [1, 1] 1
  12.19 ->   12.41 ->   13.39  [2, 1] 0
  13.37 ->   13.37 ->   14.59  [1, 1] 1
  15.17 ->   15.17 ->   15.94  [1, 0] 0
  15.18 ->   15.18 ->   16.01  [1, 1] 1
  16.74 ->   16.74 ->   17.93  [1, 0] 0
  16.85 ->   16.85 ->   18.02  [1, 1] 1
  17.20 ->   17.93 ->   18.91  [2, 1] 0
  17.54 ->   18.02 ->   18.81  [2, 2] 1
  17.85 ->   18.91 ->   20.04  [3, 2] 0
  18.52 ->   18.81 ->   19.83  [2, 2] 1
  18.68 ->   20.04 ->   20.92  [3, 2] 0
  19.57 ->   19.83 ->   20.71  [2, 2] 1
